In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from itertools import islice
import time
from random import randint
import random
from icecream import ic
from datetime import datetime
import json
from typing import List

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed,ChatGPT,ChatGPTWebsite
from Surfer.ScriptRunner import ScriptRunner
from files import *
from Stegasus import stegasus
from Bot import *
from Chatter import prompts
from Surfer.util import uuid

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros

('hi, how atre you?', '01001011111')

Setting up Typoceros Completed



In [4]:
import importlib

importlib.reload(prompts)

<module 'Chatter.prompts' from 'c:\\Users\\nasoo\\Desktop\\Y5Project\\Port\\Chatter\\prompts.py'>

In [5]:
bits_len = 200
data = stegasus.random_bit_stream(bits_len)
print(f'{bits_len // 8} Bytes')

75 Bytes


In [6]:
cache = {}

In [7]:
with open('posts.json','r') as po:
	cache['posts'] = json.loads(po.read())

In [8]:
class Encoder:
	def __init__(self,data,dataLog,encodedChatLog,identity=False):
		self.data = data
		self.rem = data
		self.dataLog = dataLog
		self.encodedChatLog = encodedChatLog
		self.dataLog.append('')
		self.sending_pure_padding = False
		self.identity = identity

	def done(self):
		return self.sending_pure_padding 

	def __call__(self,text) -> str:
		# sending pure padding
		self.sending_pure_padding = len(self.rem) < 499
		if self.identity:
			a = text
			self.rem = self.rem[60:]
		else:
			a, self.rem = stegasus.StegasusEncode(text,self.rem)
		self.encodedChatLog.append(a)
		self.dataLog.append(self.data[len(self.dataLog[-1]):-len(self.rem)])
		return a


In [9]:
# import importlib
# importlib.reload(prompts)

# backup_post = [p for p in cache['posts'] if not (p['title'] is not None and p['title'] != "" and p['excerpt'] is not None and p['excerpt'] != "")][0]
# a = prompts.inChatReplyPrompt(Message(Person('alice','nice',23,'female','Boston','alice@gmail.com'),'sup dog'),backup_post)
# print(a)

In [10]:
print(list(cache.keys()))
# del cache['data']
# del cache['driver']
# del cache['RedditFeed']
# del cache['posts']
# del cache['ChatGPTWebsite']
# del cache['encodedChatLog']
# del cache['chatLog']
# del cache['promptLog']
# del cache['dataLog']
# del cache['aliceChat']
# del cache['bobChat']
print(list(cache.keys()))

['posts']
['posts']


In [11]:
from icecream import ic

In [12]:
def tell(cache,dir_path):
  with open(f'{dir_path}/cache.json','w') as f:
    f.write(json.dumps({
      'chatLog':cache['chatLog']
      , 'encodedChatLog': cache['encodedChatLog']
      , 'promptLog': cache['promptLog']
      , 'dataLog': cache['dataLog']
      , 'data': cache['data']
      },cls=MessageEncoder,indent=2))

# define openChannelAndSend(data)

In [13]:
def openChannelAndSend(data:str
, alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
, bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')
, subreddit='unpopularopinion'
, identity=False):
	data += '1' + ('0' * 500)
	cache['data'] = data
	if 'driver' in cache:
		driver = cache['driver']
	else:
		cache['driver'] = GetBrowser()
		driver = cache['driver']
	driver.implicitly_wait(2)
	rf = RedditFeed(driver,skip=False,subreddit=subreddit,newWindow=False)
	posts = []
	if 'posts' in cache:
		posts = cache['posts']
	else:
		for x in islice(rf,15):
			posts.append(x)
			time.sleep(2*max(1,len(x) * 0.01))
		posts = [p for p in posts if bool(p['excerpt']) and bool(p['title'])]
		random.shuffle(posts)
		cache['posts'] = posts
	def take_post():
		elem = posts[randint(0,len(posts)-1)] 
		posts.remove(elem)
		return elem
	gptWebsite = cache.get('ChatGPTWebsite',ChatGPTWebsite(driver,skip=False))
	
	time.sleep(7)
	
	encodedChatLog: List[str] = []
	cache['encodedChatLog'] = encodedChatLog
	chatLog: List[Message] = []
	cache['chatLog'] = chatLog
	promptLog: List[str] = []
	cache['promptLog'] = promptLog
	dataLog: List[str] = []
	cache['dataLog'] = dataLog
	tell(cache,'.')
	
	encoder = Encoder(data,dataLog,encodedChatLog,identity)

	cache['aliceChat'] = gptWebsite.getLandingNewChat()
	aliceChat = cache['aliceChat'] 

	# alice first message
	promptLog.append(prompts.start_conversation_with_post(alice,bob,take_post()))
	tell(cache,'.')
	chatLog.append(Message(alice,aliceChat.ask(promptLog[-1])))
	tell(cache,'.')
	encoder(chatLog[-1].text)
	tell(cache,'.')

	# bob first message
	promptLog.append(prompts.receiverStartChatReplyPrompt(Message(person=alice,text=encodedChatLog[-1])))
	tell(cache,'.')
	a,bobChat = aliceChat.askNew(promptLog[-1])
	tell(cache,'.')
	chatLog.append(Message(bob,a))
	tell(cache,'.')

	# alice second message
	promptLog.append(prompts.senderStartChatReplyPrompt(chatLog[-1]))
	tell(cache,'.')
	chatLog.append(Message(alice, aliceChat.ask(promptLog[-1])))
	tell(cache,'.')
	a = encoder(chatLog[-1].text)
	tell(cache,'.')
	alice_message_number = 2

	while True:
		if encoder.done():
			promptLog.append(prompts.endChatReplyPrompt(chatLog[-1]))
			tell(cache,'.')
			chatLog.append(Message(alice, aliceChat.ask(promptLog[-1])))
			tell(cache,'.')
			a = encoder(chatLog[-1].text)
			tell(cache,'.')
			return None

		promptLog.append(prompts.receiverInChatReplyPrompt(Message(alice,encodedChatLog[-1])))
		tell(cache,'.')
		chatLog.append(Message(bob, bobChat.ask(promptLog[-1])))
		tell(cache,'.')
		
		# switch
		alice_message_number += 1
		promptLog.append(prompts.senderInChatReplyPrompt(chatLog[-1],take_post() if alice_message_number % 7 == 0 else None))
		tell(cache,'.')
		chatLog.append(Message(alice, aliceChat.ask(promptLog[-1])))
		tell(cache,'.')
		a = encoder(chatLog[-1].text)
		tell(cache,'.')

# run it

In [14]:
openChannelAndSend(data,subreddit='CasualConversation')

In [15]:
tell(cache,'.')

In [16]:
from json import dumps
import pandas as pd
import shutil
import json
import subprocess
import os

def augment_stats():
  df = pd.read_csv('./stats.tsv',delimiter='\t')
  df['capacity'] = df['capacity'].apply(lambda x : round(100*x) if isinstance(x,float) else x)
  df.to_csv('./stats.tsv',sep='\t',index=False)
  dfd = df.describe()
  dfd['bits'] = dfd['bits'].apply(lambda a: int(round(a)))
  dfd['cover size'] = dfd['cover size'].apply(lambda a: int(round(a)))
  dfd['capacity'] = dfd['capacity'].apply(lambda a: int(round(a)))
  dfd.to_csv('./stats.stats.tsv',sep='\t')

def render(path,image=None):
  with open(path,'r') as fff:
    cache = json.loads(fff.read())
  stats_path = path[:-len('cache.json')]+'stats.tsv'
  stats = pd.read_csv(stats_path,delimiter='\t')
  stats_stats = pd.read_csv(path[:-len('cache.json')]+'stats.stats.tsv',delimiter='\t')
  stats_stats.rename(columns={'Unnamed: 0':''},inplace=True)
  normal_chat = json.dumps(cache['chatLog'])
  encoded_chat = []
  aliceChatLog: List[str] = cache['encodedChatLog']
  def f(i,m):
    if len(aliceChatLog) > 0:
      m['text'] = aliceChatLog[0]
      aliceChatLog.remove(m['text'])
    return m
  aliceChatLog = list(f(i,m) for i, m in enumerate(cache['chatLog']) if m['person'] == 'alice nice')
  bobChatLog = list(m for m in cache['chatLog'] if m['person'] == 'bob wick')
  for i in range(max(len(aliceChatLog),len(bobChatLog))):
    if i < len(aliceChatLog):
      encoded_chat.append(aliceChatLog[i])
    if i < len(bobChatLog):
      encoded_chat.append(bobChatLog[i])
  encoded_chat = json.dumps(encoded_chat)
  encoded_chat_data = json.dumps([{'bits':x[0],'cover size':x[1],'capacity':f'{x[2]}%'} for x in stats.to_numpy().tolist()])
  summary = stats_stats.to_html(index=False)
  image_details = '' 
  if not image is None:
    with open(image+'.html','r') as ff:
      image_details = ff.read()
    print(image_details)
  image = '' if image is None else f'<img src="file://{os.path.abspath(image)}" style="width: fit-content;height: fit-content;scale: 25;margin: 7rem;"/>'
  with open('./chat.template.html','r') as fff, open(f'{path}.html','w') as o:
    o.write(fff.read().replace('$normal_chat',normal_chat)
    .replace('$encoded_chat_data',encoded_chat_data)
    .replace('$encoded_chat',encoded_chat)
    .replace('$summary',summary)
    .replace('$image_details',image_details)
    .replace('$image',image)
    )
  file_path = os.path.abspath(f'{path}.html'.replace('/','\\'))
  print(file_path)
  try:
      subprocess.Popen(['start', '', file_path], shell=True)
      print(f"Opened {file_path} in the default application.")
  except Exception as e:
      print(f"Error: {e}")

def saveCache(cache=cache):
  id = datetime.now().__str__().replace(':','-')[:19]
  tell(cache,'.')
  dir_path = f'benchmarks_spans_5/{id}'
  os.mkdir(dir_path)

  augment_stats()
  shutil.move('stats.tsv', dir_path+'/stats.tsv')
  shutil.move('cache.json', dir_path+'/cache.json')
  shutil.move('stats.stats.tsv', dir_path+'/stats.stats.tsv')
  shutil.move('steps.log', dir_path+'/steps.log')
  shutil.move('pie.tsv', dir_path+'/pie.tsv')
  render(f'{dir_path}/cache.json')

saveCache()


c:\Users\nasoo\Desktop\Y5Project\Port\benchmarks_spans_5\2023-08-13 13-11-14\cache.json.html
Opened c:\Users\nasoo\Desktop\Y5Project\Port\benchmarks_spans_5\2023-08-13 13-11-14\cache.json.html in the default application.


In [17]:
render('benchmarks_spans_5/2023-08-13 12-43-09/cache.json')

c:\Users\nasoo\Desktop\Y5Project\Port\benchmarks_spans_5\2023-08-13 12-43-09\cache.json.html
Opened c:\Users\nasoo\Desktop\Y5Project\Port\benchmarks_spans_5\2023-08-13 12-43-09\cache.json.html in the default application.


In [13]:
import pandas as pd
# df = pd.read_csv('./examples/ZcEybAIIxHKeeuiqUapd/pie.tsv',delimiter='\t')
fs = ['12-43-09','13-11-14']
for f in fs:
	df = pd.read_csv(f'./benchmarks_spans_5/2023-08-13 {f}/pie.tsv',delimiter='\t')
	df['MLM capacity'] = df['MLM capacity'].apply(lambda x : round(100*x) if isinstance(x,float) else x)
	df['Typo capacity'] = df['Typo capacity'].apply(lambda x : round(100*x) if isinstance(x,float) else x)
	df['Emo capacity'] = df['Emo capacity'].apply(lambda x : round(100*x) if isinstance(x,float) else x)
	df.columns = df.columns.astype(str)
	typo = df[['cover size', 'Typo bits','Typo capacity']]
	typo = typo.rename({ 
		'cover size': "cover size"
		,'Typo bits': "bits"
		,'Typo capacity': "ratio"
	})
	emo = df[['cover size', 'Emo bits','Emo capacity']]
	emo = emo.rename({ 
		'cover size': "cover size"
		,'Emo bits': "bits"
		,'Emo capacity': "ratio"
	})
	typo.to_csv(f'./d typo benchmark{f}.tsv',sep='\t',index=False)
	emo.to_csv(f'./d emoji benchmark{f}.tsv',sep='\t',index=False)